# Rolling sum - Pandas vs Numpy

In [ ]:
import numpy as np
import pandas as pd
from time import process_time_ns
from numpy.lib.stride_tricks import as_strided

#### Define functions and prepare data

In [ ]:
def rolling_sum(a, n) :
    ret = np.cumsum(a, axis=0, dtype=float)
    #ret[:, n:] = ret[:, n:] - ret[:, :-n]
    #return ret[:, n - 1:]
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:]

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return as_strided(a, shape=shape, strides=strides)

In [ ]:
dt = pd.date_range('1/1/2000', periods=200)
ts = np.random.randn(200)
df = pd.DataFrame(ts, index=dt)
w = 20

## Comparison

In [ ]:
%%timeit
for i in range(100):
    pd_res = df.rolling(w, min_periods=1).sum()

In [ ]:
%%timeit
for i in range(100):
    cs_res = rolling_sum(ts, w)

In [ ]:
%%timeit
for i in range(100):
    rw_res = np.sum(rolling_window(ts, w), 0)

#### Check correctnes

In [ ]:
pd_res = df.rolling(w, min_periods=w).sum()
cs_res = rolling_sum(ts, w)
rw_res = np.sum(rolling_window(ts, w), 0)

In [ ]:
pd_res.values.shape, cs_res.shape
np.sum(np.ravel(pd_res.values[w-1:])-cs_res)